In [ ]:
# Grabs the data from ACTT database and exports it to Excel files


import os
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime

DB_HOST = "aact-db.ctti-clinicaltrials.org"
DB_PORT = 5432
DB_NAME = "aact"
DB_USER = "USERHERE"
DB_PASS = "PASSHERE"

# Define the path where you want to save the Excel files
OUTPUT_PATH = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"

connection_url = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_url)

def export_filtered_data():
    condition_filter = """
        c.name ILIKE '%breast cancer%'
        OR c.name ILIKE '%type 2 diabetes%'
        OR c.name ILIKE '%alzheimer%'
    """
    table_queries = {
        'studies': text(f"""
            SELECT DISTINCT st.*
            FROM studies st
            JOIN conditions c ON st.nct_id = c.nct_id
            WHERE {condition_filter};
        """),
        'sponsors': text(f"""
            SELECT DISTINCT sp.*
            FROM sponsors sp
            JOIN studies st ON sp.nct_id = st.nct_id
            JOIN conditions c ON st.nct_id = c.nct_id
            WHERE {condition_filter};
        """),
        'design_outcomes': text(f"""
            SELECT DISTINCT d_out.*
            FROM design_outcomes d_out
            JOIN studies st ON d_out.nct_id = st.nct_id
            JOIN conditions c ON st.nct_id = c.nct_id
            WHERE {condition_filter};
        """),
        'facilities': text(f"""
            SELECT DISTINCT fa.*
            FROM facilities fa
            JOIN studies st ON fa.nct_id = st.nct_id
            JOIN conditions c ON st.nct_id = c.nct_id
            WHERE {condition_filter};
        """),
        'eligibilities': text(f"""
            SELECT DISTINCT el.*
            FROM eligibilities el
            JOIN studies st ON el.nct_id = st.nct_id
            JOIN conditions c ON st.nct_id = c.nct_id
            WHERE {condition_filter};
        """),
        'interventions': text(f"""
            SELECT DISTINCT i.*
            FROM interventions i
            JOIN studies st ON i.nct_id = st.nct_id
            JOIN conditions c ON st.nct_id = c.nct_id
            WHERE {condition_filter};
        """)
    }

    try:
        with engine.connect() as conn:
            for table_name, query in table_queries.items():
                df = pd.read_sql(query, conn)
                output_filename = os.path.join(OUTPUT_PATH, f"{table_name}.xlsx")
                df.to_excel(output_filename, index=False)
                print(f"Exported {table_name}.xlsx with {len(df)} rows to {OUTPUT_PATH}.")

            columns_for_llm = [
                "nct_id",
                "parsed_keywords",
                "category_tags",
                "summarized_text",
                "created_at"
            ]
            df_llm = pd.DataFrame(columns=columns_for_llm)

            llm_filename = os.path.join(OUTPUT_PATH, "LLM_as_a_parser.xlsx")
            df_llm.to_excel(llm_filename, index=False)
            print(f"Created placeholder LLM data Excel: {llm_filename}")

    except Exception as e:
        print(f"Error querying AACT database or exporting data: {e}")


In [ ]:
# Set up Local DeepSeek LLM Environment

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

model_path = "deepseek-ai/deepseek-llm-7b-chat"
device = "mps"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=dtype,
    device_map=device,
    trust_remote_code=True,
)
streamer = TextStreamer(tokenizer, skip_prompt=True)

prompt = """
<|im_start|>system
You are a helpful assistant that answers questions.
<|im_end|>
<|im_start|>user
What is the capital of France?
<|im_end|>
<|im_start|>assistant
"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.95,
        top_k=60,
        temperature=0.2,
        num_return_sequences=1,
    )

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"Model Output:\n{decoded_output}")

In [ ]:
# Token Cost

import os
import pandas as pd
import tiktoken

PRICE_USD_PER_1K_TOKENS_PROMPT = 0.03
PRICE_USD_PER_1K_TOKENS_COMPLETION = 0.06

ESTIMATED_COMPLETION_TOKENS = 200

SYSTEM_PROMPT = """You are an advanced AI that parses the clinical trial eligibility criteria text.
You will:
1) Summarize the major inclusion criteria in bullet points
2) Summarize the major exclusion criteria in bullet points
3) Identify any special flags or conditions (e.g. 'BRCA mutation', 'HER2', 'Stage IV', 'No prior chemo')

Return your output as valid JSON with fields:
{
  "inclusion_points": [...],
  "exclusion_points": [...],
  "special_flags": [...]
}
"""

USER_PROMPT_TEMPLATE = """Please analyze the following 'eligibility criteria' text and generate JSON accordingly:

Text to analyze:
{TEXT}"""

def build_prompt(text_block):
    """
    Builds a hypothetical ChatCompletion message list
    with system + user instructions for GPT-4.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT_TEMPLATE.format(TEXT=text_block)}
    ]
    return messages

def num_tokens_from_messages(messages, model="gpt-4"):
    """
    Approximate token usage for GPT-4 by using 'gpt-3.5-turbo' encoding
    (closest known approach).
    """
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = 0
    for msg in messages:
        num_tokens += 4
        for key, value in msg.items():
            num_tokens += len(encoding.encode(value))
    num_tokens += 2
    return num_tokens

def main():
    DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
    excel_file = os.path.join(DATA_FOLDER, "eligibilities.xlsx")
    column_with_text = "criteria"

    df = pd.read_excel(excel_file)
    print(f"Loaded {len(df)} rows from {excel_file}.")

    total_prompt_tokens = 0
    total_calls = 0

    for idx, row in df.iterrows():
        text_col = row.get(column_with_text, "")
        if not isinstance(text_col, str) or not text_col.strip():
            continue

        messages = build_prompt(text_col)
        prompt_tokens = num_tokens_from_messages(messages, model="gpt-4")

        total_prompt_tokens += prompt_tokens
        total_calls += 1

    print(f"Number of rows with non-empty '{column_with_text}': {total_calls}")
    print(f"Total prompt tokens (input): {total_prompt_tokens}")

    total_completion_tokens = total_calls * ESTIMATED_COMPLETION_TOKENS
    print(f"Estimated total completion tokens (output): {total_completion_tokens}")

    prompt_tokens_k = total_prompt_tokens / 1000
    completion_tokens_k = total_completion_tokens / 1000

    cost_prompt = prompt_tokens_k * PRICE_USD_PER_1K_TOKENS_PROMPT
    cost_completion = completion_tokens_k * PRICE_USD_PER_1K_TOKENS_COMPLETION
    total_cost = cost_prompt + cost_completion

    print("----- COST ESTIMATE -----")
    print(f"Prompt tokens cost:     ${cost_prompt:.2f} (for ~{prompt_tokens_k:.2f}k tokens)")
    print(f"Completion tokens cost: ${cost_completion:.2f} (for ~{completion_tokens_k:.2f}k tokens)")
    print(f"Total estimated cost:   ${total_cost:.2f}")


In [ ]:
# DeepSeek LM Locally Run

import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# ------------------------------------------------
# 1) Configuration
# ------------------------------------------------
MODEL_PATH = "deepseek-ai/deepseek-llm-7b-chat"
DEVICE = "mps"
DTYPE = torch.bfloat16
NUM_ROWS = 10

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
EXCEL_INPUT = os.path.join(DATA_FOLDER, "eligibilities.xlsx")
EXCEL_OUTPUT = os.path.join(DATA_FOLDER, "LLM_as_a_parser.xlsx")

# ------------------------------------------------
# 2) System & User Prompts
# ------------------------------------------------
SYSTEM_PROMPT = """You are an AI that parses clinical trial eligibility criteria into specific fields.
For each criteria text, identify and extract the following in valid JSON only:

1) performance_status: e.g. "ECOG 0-1" or "KPS ≥70" if mentioned, or "N/A" if absent
2) biomarkers: a list of relevant markers (e.g. "BRCA", "HER2_neg", "EGFR", "ER_neg")
3) no_prior_chemo: boolean, true if text excludes participants with prior chemo, otherwise false
4) pregnancy_excluded: boolean, true if pregnant/breastfeeding women are excluded, otherwise false
5) excluded_comorbidities: array of major excluded diseases/conditions (e.g. "uncontrolled seizures", "HIV infection")
6) other_special_requirements: array for any additional must-have or must-not-have conditions (e.g., "must have tissue blocks", "no major surgery within 14 days")

Return valid JSON only, with no extra text or special characters (like code fences). For example:
{
  "performance_status": "ECOG 0-1",
  "biomarkers": ["BRCA", "HER2_neg"],
  "no_prior_chemo": false,
  "pregnancy_excluded": true,
  "excluded_comorbidities": ["uncontrolled seizures", "heart failure"],
  "other_special_requirements": ["tumor blocks", "no major surgery within 14 days"]
}"""

USER_PROMPT_TEMPLATE = """Below is the full eligibility criteria for a clinical trial:

{CRITERIA}

Please parse it into the requested fields (performance_status, biomarkers, etc.)."""

# ------------------------------------------------
# 3) Load Model
# ------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=DTYPE,
    device_map=DEVICE,
    trust_remote_code=True,
)

def generate_response(system_prompt, user_prompt, max_new_tokens=768):
    combined_prompt = f"""
<|im_start|>system
{system_prompt}
<|im_end|>

<|im_start|>user
{user_prompt}
<|im_end|>

<|im_start|>assistant
"""
    inputs = tokenizer(combined_prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.3,
        )
    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

def main():
    # ------------------------------------------------
    # 4) Load data and limit to first 10 rows
    # ------------------------------------------------
    df = pd.read_excel(EXCEL_INPUT)
    print(f"Loaded {len(df)} rows from {EXCEL_INPUT}. Only processing first {NUM_ROWS} rows.")
    df = df.head(NUM_ROWS)

    results = []

    for idx, row in df.iterrows():
        nct_id = row.get("nct_id", "")
        criteria_text = row.get("criteria", "")

        if not isinstance(criteria_text, str) or not criteria_text.strip():
            continue

        user_prompt = USER_PROMPT_TEMPLATE.format(CRITERIA=criteria_text)
        raw_output = generate_response(SYSTEM_PROMPT, user_prompt)

        start = raw_output.find("{")
        end = raw_output.rfind("}")
        if start != -1 and end != -1 and end > start:
            json_substring = raw_output[start:end+1]
        else:
            json_substring = raw_output

        results.append({
            "nct_id": nct_id,
            "criteria_json_substring": json_substring,
            "full_raw_response": raw_output
        })

    # ------------------------------------------------
    # 5) Save to Excel
    # ------------------------------------------------
    out_df = pd.DataFrame(results)
    out_df.to_excel(EXCEL_OUTPUT, index=False)
    print(f"Done. Saved results to {EXCEL_OUTPUT}")


In [ ]:
# GPT 4 API

import os
import openai
import pandas as pd

# ------------------------------
# 1) Configuration
# ------------------------------
openai.api_key = "Key Here"


MODEL_NAME = "gpt-4"
NUM_ROWS = 750

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
EXCEL_INPUT = os.path.join(DATA_FOLDER, "eligibilities.xlsx")
EXCEL_OUTPUT = os.path.join(DATA_FOLDER, "LLM_as_a_parser.xlsx")

# ------------------------------
# 2) Prompts
# ------------------------------
SYSTEM_PROMPT = """You are an advanced AI that extracts key fields from unstructured clinical trial eligibility criteria
and returns them in valid JSON. If a field is not found, set it to 'N/A', false, or [].

We want these fields in the JSON:
1) performance_status (e.g., "ECOG 0-1" or "KPS >=70" or "N/A")
2) biomarkers (array, e.g., ["BRCA"], or [])
3) no_prior_chemo (boolean)
4) pregnancy_excluded (boolean)
5) excluded_comorbidities (array of conditions)
6) other_special_requirements (array)

Return **only** valid JSON from the first '{' to the last '}', with no extra text.
If you see disclaimers or code fences, remove them.

### One-Shot Example

Example Criteria:
"Inclusion:
- ECOG 0-1
Exclusion:
- No prior chemo, uncontrolled seizures
"

Example JSON:
{
  "performance_status": "ECOG 0-1",
  "biomarkers": [],
  "no_prior_chemo": true,
  "pregnancy_excluded": false,
  "excluded_comorbidities": ["uncontrolled seizures"],
  "other_special_requirements": []
}
"""

USER_PROMPT_TEMPLATE = """Below is the full eligibility criteria for a clinical trial:

{CRITERIA}

Return these 6 fields in valid JSON, from the first '{{' to the last '}}' only.
"""


# ------------------------------
# 3) GPT-4 API Call
# ------------------------------
def call_gpt4_api(system_prompt, user_prompt):
    """
    Uses openai>=1.0.0 style chat.completions.create for GPT-4.
    """
    response = openai.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
        max_tokens=800,
        top_p=0.9,
    )
    return response.choices[0].message.content

def extract_json_substring(full_text):
    """
    Extract the substring from the first '{' to the last '}'.
    If no braces found, return the entire text for fallback.
    """
    start = full_text.find("{")
    end = full_text.rfind("}")
    if start != -1 and end != -1 and end > start:
        return full_text[start:end+1]
    else:
        return full_text

# ------------------------------
# 4) Main Script
# ------------------------------
def main():
    df = pd.read_excel(EXCEL_INPUT)
    print(f"Loaded {len(df)} rows from {EXCEL_INPUT}. Only processing first {NUM_ROWS} rows.")
    df = df.head(NUM_ROWS)

    results = []

    for idx, row in df.iterrows():
        nct_id = row.get("nct_id", "")
        criteria_text = row.get("criteria", "")

        if not isinstance(criteria_text, str) or not criteria_text.strip():
            continue

        user_prompt = USER_PROMPT_TEMPLATE.format(CRITERIA=criteria_text)

        raw_output = call_gpt4_api(SYSTEM_PROMPT, user_prompt)

        json_substring = extract_json_substring(raw_output)

        results.append({
            "nct_id": nct_id,
            "full_raw_response": raw_output,
            "criteria_json_substring": json_substring
        })

    out_df = pd.DataFrame(results)
    out_df.to_excel(EXCEL_OUTPUT, index=False)
    print(f"Done. Saved results to {EXCEL_OUTPUT}")



In [ ]:
# DeepSeek Distill R1

import os
import torch
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ------------------------------
# 1) Configuration
# ------------------------------
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
EXCEL_INPUT = os.path.join(DATA_FOLDER, "eligibilities.xlsx")
EXCEL_OUTPUT = os.path.join(DATA_FOLDER, "LLM_as_a_parser_DeepSeek_Distill_Qwen.xlsx")

NUM_ROWS = 10
BATCH_SIZE = 5

SYSTEM_INSTRUCTIONS = """You are an AI that extracts key fields from unstructured clinical trial eligibility text
and returns them in valid JSON only. If not mentioned, set "N/A", false, or [].

We want the following fields:
1) performance_status (e.g., "ECOG 0-1" or "N/A")
2) biomarkers (array, e.g. ["BRCA"], or [])
3) no_prior_chemo (boolean)
4) pregnancy_excluded (boolean)
5) excluded_comorbidities (array)
6) other_special_requirements (array)

Return only valid JSON from the first '{' to the last '}'—no disclaimers, code fences, or additional text.
"""

USER_PROMPT_TEMPLATE = """Below is the full eligibility criteria for a clinical trial:

{CRITERIA}

Return the 6 fields in valid JSON, from the first '{{' to the last '}}' only.
"""

def build_prompt(system_instructions, user_text):
    """
    Many Hugging Face LLMs expect a single text prompt
    (unless they are specifically chat-based).
    We'll just concatenate: system instructions + user text.
    """
    final_prompt = f"[SYSTEM]\n{system_instructions}\n\n[USER]\n{user_text}\n\n[ASSISTANT]"
    return final_prompt

def extract_json_substring(full_text):
    start = full_text.find("{")
    end = full_text.rfind("}")
    if start != -1 and end != -1 and end > start:
        return full_text[start:end+1]
    else:
        return full_text

def main():
    # ------------------------------
    # 2) Load the Distilled Qwen 1.5B Model
    # ------------------------------
    print(f"Loading model from Hugging Face: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16
    )

    generate_text = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=2048,
        temperature=0.2,
        do_sample=False
    )

    # ------------------------------
    # 3) Read Input Data
    # ------------------------------
    df = pd.read_excel(EXCEL_INPUT)
    df = df.head(NUM_ROWS)
    results = []

    batch_start = 0
    total = len(df)

    while batch_start < total:
        batch_end = min(batch_start + BATCH_SIZE, total)
        batch_df = df.iloc[batch_start:batch_end]

        for idx, row in batch_df.iterrows():
            nct_id = row.get("nct_id", "")
            criteria_text = row.get("criteria", "")
            if not isinstance(criteria_text, str) or not criteria_text.strip():
                continue

            user_prompt = USER_PROMPT_TEMPLATE.format(CRITERIA=criteria_text)
            final_prompt = build_prompt(SYSTEM_INSTRUCTIONS, user_prompt)

            outputs = generate_text(
                final_prompt,
                max_new_tokens=512,
                num_return_sequences=1
            )
            raw_output = outputs[0]["generated_text"]

            json_substring = extract_json_substring(raw_output)

            results.append({
                "nct_id": nct_id,
                "full_raw_response": raw_output,
                "criteria_json_substring": json_substring
            })

        out_df = pd.DataFrame(results)
        out_df.to_excel(EXCEL_OUTPUT, index=False)
        print(f"Processed rows {batch_start} to {batch_end-1}, partial results in {EXCEL_OUTPUT}")

        batch_start = batch_end

    print("All done, final saved.")



In [ ]:
# GPT 4 Through Azure

import os
import pandas as pd
from openai import AzureOpenAI

# Environment/config variables
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Instantiate the AzureOpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

NUM_ROWS = 500
BATCH_SIZE = 5

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
EXCEL_INPUT = os.path.join(DATA_FOLDER, "eligibilities.xlsx")
EXCEL_OUTPUT = os.path.join(DATA_FOLDER, "LLM_as_a_parser.xlsx")

# Prompt instructions
SYSTEM_PROMPT = """You are an advanced AI that extracts key fields from unstructured clinical trial eligibility text
and returns them in valid JSON only. If not mentioned, set 'N/A', false, or [].

We want these fields:
1) performance_status (e.g. "ECOG 0-1" or "N/A")
2) biomarkers (array, e.g. ["BRCA"], or [])
3) no_prior_chemo (boolean)
4) pregnancy_excluded (boolean)
5) excluded_comorbidities (array)
6) other_special_requirements (array)

Return only valid JSON from the first '{' to the last '}' with no disclaimers.
"""

USER_PROMPT_TEMPLATE = """Below is the full eligibility criteria for a clinical trial:

{CRITERIA}

Return the 6 fields in valid JSON, from the first '{{' to the last '}}' only.
"""

def call_azure_openai_api(system_prompt, user_prompt):
    """
    Calls the AzureOpenAI client for a chat completion using your GPT-4 deployment.
    """
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2,
        max_tokens=800,
        top_p=0.9
    )
    return response.choices[0].message.content

def main():
    df = pd.read_excel(EXCEL_INPUT)
    df = df.head(NUM_ROWS)
    results = []

    total_rows = len(df)
    batch_start = 0

    while batch_start < total_rows:
        batch_end = min(batch_start + BATCH_SIZE, total_rows)
        batch_df = df.iloc[batch_start:batch_end]

        for idx, row in batch_df.iterrows():
            nct_id = row.get("nct_id", "")
            criteria_text = row.get("criteria", "")

            if not isinstance(criteria_text, str) or not criteria_text.strip():
                continue

            user_prompt = USER_PROMPT_TEMPLATE.format(CRITERIA=criteria_text)
            raw_output = call_azure_openai_api(SYSTEM_PROMPT, user_prompt)

            results.append({
                "nct_id": nct_id,
                "full_raw_response": raw_output
            })

        out_df = pd.DataFrame(results)
        out_df.to_excel(EXCEL_OUTPUT, index=False)
        print(f"Processed rows {batch_start} to {batch_end-1}, partial results -> {EXCEL_OUTPUT}")

        batch_start = batch_end

    print("All done, final results saved.")



In [ ]:
# Parsing JSON Output

import os
import re
import json
import pandas as pd

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
INPUT_FILE = os.path.join(DATA_FOLDER, "LLM_as_a_parser.xlsx")
OUTPUT_FILE = os.path.join(DATA_FOLDER, "LLM_as_a_parser_parsed_values.xlsx")

def parse_json_fields():
    df = pd.read_excel(INPUT_FILE)

    extracted_rows = []

    for idx, row in df.iterrows():
        nct_id = row["nct_id"]
        raw_response = str(row["full_raw_response"])

        cleaned_response = re.sub(r"^```(?:json)?", "", raw_response.strip(), flags=re.IGNORECASE).strip()
        cleaned_response = re.sub(r"```$", "", cleaned_response, flags=re.IGNORECASE).strip()

        try:
            data = json.loads(cleaned_response)

            fields = [
                "performance_status",
                "biomarkers",
                "no_prior_chemo",
                "pregnancy_excluded",
                "excluded_comorbidities",
                "other_special_requirements"
            ]

            for field_name in fields:
                field_value = data.get(field_name, None)
                extracted_rows.append({
                    "nct_id": nct_id,
                    "raw_response": raw_response,
                    "field_name": field_name,
                    "field_value": field_value
                })

        except json.JSONDecodeError:
            extracted_rows.append({
                "nct_id": nct_id,
                "raw_response": raw_response,
                "field_name": "error",
                "field_value": "JSON parse error"
            })

    parsed_df = pd.DataFrame(extracted_rows)

    parsed_df.to_excel(OUTPUT_FILE, index=False)
    print(f"Parsed results saved to {OUTPUT_FILE}")



In [ ]:
# Grab Conditions Table

DB_HOST = "aact-db.ctti-clinicaltrials.org"
DB_PORT = 5432
DB_NAME = "aact"
DB_USER = "USERHERE"
DB_PASS = "PASSHERE"

OUTPUT_PATH = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"

os.makedirs(OUTPUT_PATH, exist_ok=True)

connection_url = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_url)

def export_conditions_data():
    """
    Connects to the AACT database, filters for trials mentioning
    'Breast Cancer', 'Type 2 Diabetes', or 'Alzheimer'
    in the conditions.name field, and exports all columns (*) from
    the 'conditions' table to a single Excel file in OUTPUT_PATH.
    """

    condition_filter = """
        c.name ILIKE '%breast cancer%'
        OR c.name ILIKE '%type 2 diabetes%'
        OR c.name ILIKE '%alzheimer%'
    """

    conditions_query = text(f"""
        SELECT DISTINCT c.*
        FROM conditions c
        JOIN studies st ON c.nct_id = st.nct_id
        WHERE {condition_filter};
    """)

    try:
        with engine.connect() as conn:
            df_conditions = pd.read_sql(conditions_query, conn)
            output_file = os.path.join(OUTPUT_PATH, "conditions.xlsx")
            df_conditions.to_excel(output_file, index=False)
            print(f"Exported conditions.xlsx with {len(df_conditions)} rows to {OUTPUT_PATH}.")

    except Exception as e:
        print(f"Error exporting conditions table: {e}")



In [ ]:
# Pivot Table

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
INPUT_FILE = os.path.join(DATA_FOLDER, "LLM_as_a_parser.xlsx")
OUTPUT_FILE = os.path.join(DATA_FOLDER, "LLM_as_a_parser_parsed_values_pivoted.xlsx")

def parse_json_fields_and_pivot():
    df = pd.read_excel(INPUT_FILE)

    extracted_rows = []

    for idx, row in df.iterrows():
        nct_id = row["nct_id"]
        raw_response = str(row["full_raw_response"])

        cleaned_response = re.sub(r"^```(?:json)?", "", raw_response.strip(), flags=re.IGNORECASE).strip()
        cleaned_response = re.sub(r"```$", "", cleaned_response, flags=re.IGNORECASE).strip()

        try:
            data = json.loads(cleaned_response)

            fields = [
                "performance_status",
                "biomarkers",
                "no_prior_chemo",
                "pregnancy_excluded",
                "excluded_comorbidities",
                "other_special_requirements"
            ]

            for field_name in fields:
                field_value = data.get(field_name, None)
                extracted_rows.append({
                    "nct_id": nct_id,
                    "raw_response": raw_response,
                    "field_name": field_name,
                    "field_value": field_value
                })

        except json.JSONDecodeError:
            extracted_rows.append({
                "nct_id": nct_id,
                "raw_response": raw_response,
                "field_name": "error",
                "field_value": "JSON parse error"
            })

    tall_df = pd.DataFrame(extracted_rows)

    latest_raw = (
        tall_df.groupby("nct_id")["raw_response"]
        .last()  # or .first()
        .reset_index()
    )

    pivoted = tall_df.pivot(index="nct_id", columns="field_name", values="field_value")

    pivoted = pivoted.reset_index().merge(latest_raw, on="nct_id", how="left")

    pivoted.to_excel(OUTPUT_FILE, index=False)
    print(f"Pivoted results saved to {OUTPUT_FILE}")



In [ ]:
# AI Outcomes Extraction

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Instantiate the AzureOpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Adjust as needed
NUM_ROWS = 500
BATCH_SIZE = 5

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
EXCEL_INPUT = os.path.join(DATA_FOLDER, "design_outcomes.xlsx")
EXCEL_OUTPUT = os.path.join(DATA_FOLDER, "ai_outcomes.xlsx")


SYSTEM_PROMPT = """You are an advanced AI that extracts key fields from unstructured clinical trial outcome text
and returns them in valid JSON only. If not mentioned, set 'N/A' or [].

We want these fields:
1) key_variables (array/list of strings) - e.g. ["HbA1c","Blood Pressure"]
2) time_frame (string) - e.g. "12 weeks","Up to 5 years", or "N/A"
3) outcome_category (string) - e.g. "Survival","Quality of Life","Biomarker","Safety","Other"

Return only valid JSON from the first '{' to the last '}' with no disclaimers.
"""

USER_PROMPT_TEMPLATE = """Below is an outcome measure text for a clinical trial:

{OUTCOME_TEXT}

Return 3 fields in valid JSON, from the first '{{' to the last '}}' only:
- key_variables (array)
- time_frame (string)
- outcome_category (string)
"""

def call_azure_openai_api(system_prompt, user_prompt):
    """
    Calls the AzureOpenAI client for a chat completion using your GPT-4 deployment.
    """
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2,
        max_tokens=800,
        top_p=0.9
    )
    return response.choices[0].message.content

def main():
    df = pd.read_excel(EXCEL_INPUT)
    df = df.head(NUM_ROWS)
    results = []

    total_rows = len(df)
    batch_start = 0

    while batch_start < total_rows:
        batch_end = min(batch_start + BATCH_SIZE, total_rows)
        batch_df = df.iloc[batch_start:batch_end]

        for idx, row in batch_df.iterrows():
            nct_id = row.get("nct_id", "")
            text_for_llm = row.get("measure", "")

            if not isinstance(text_for_llm, str) or not text_for_llm.strip():
                continue

            user_prompt = USER_PROMPT_TEMPLATE.format(OUTCOME_TEXT=text_for_llm)
            raw_output = call_azure_openai_api(SYSTEM_PROMPT, user_prompt)

            results.append({
                "nct_id": nct_id,
                "full_raw_response": raw_output
            })

        out_df = pd.DataFrame(results)
        out_df.to_excel(EXCEL_OUTPUT, index=False)
        print(f"Processed rows {batch_start} to {batch_end - 1}, partial results -> {EXCEL_OUTPUT}")

        batch_start = batch_end

    print("All done, final results saved to:", EXCEL_OUTPUT)



In [ ]:
# AI Intervention Extraction

# Azure OpenAI environment/config variables
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Instantiate the AzureOpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Adjust these as needed
NUM_ROWS = 500
BATCH_SIZE = 5

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
EXCEL_INPUT = os.path.join(DATA_FOLDER, "interventions.xlsx")
EXCEL_OUTPUT = os.path.join(DATA_FOLDER, "ai_interventions.xlsx")


SYSTEM_PROMPT = """You are an advanced AI that extracts structured info from unstructured clinical trial interventions.
Return only valid JSON. If data is not mentioned, use 'N/A' or an empty array/list.

We want these 5 fields:
1) drug_names (array of strings) - e.g. ["Carboplatin","Paclitaxel"]
2) dosages (array of strings) - e.g. ["80 mg/m2 weekly","AUC=5","200 mg once daily"]
3) administration_route (string) - e.g. "Intravenous","Oral","N/A"
4) frequency (string) - e.g. "Every 3 weeks for 4 cycles","Once daily","N/A"
5) therapy_class (string) - e.g. "Chemotherapy","Immunotherapy","Hormonal","Targeted Therapy","Supportive Care","Other"

Return only valid JSON from the first '{' to the last '}' with no disclaimers.
"""

USER_PROMPT_TEMPLATE = """Below is the intervention description for a clinical trial:

{INTERVENTION_TEXT}

Please return the 5 fields in valid JSON, from the first '{{' to the last '}}'.
"""

def call_azure_openai_api(system_prompt, user_prompt):
    """
    Calls the AzureOpenAI client for a chat completion using your GPT-4 deployment.
    """
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2,
        max_tokens=800,
        top_p=0.9
    )
    return response.choices[0].message.content

def main():
    df = pd.read_excel(EXCEL_INPUT)
    df = df.head(NUM_ROWS)

    results = []
    total_rows = len(df)
    batch_start = 0

    while batch_start < total_rows:
        batch_end = min(batch_start + BATCH_SIZE, total_rows)
        batch_df = df.iloc[batch_start:batch_end]

        for idx, row in batch_df.iterrows():
            nct_id = row.get("nct_id", "")
            intervention_text = row.get("description", "")

            if not isinstance(intervention_text, str) or not intervention_text.strip():
                continue

            user_prompt = USER_PROMPT_TEMPLATE.format(INTERVENTION_TEXT=intervention_text)
            raw_output = call_azure_openai_api(SYSTEM_PROMPT, user_prompt)

            results.append({
                "nct_id": nct_id,
                "full_raw_response": raw_output
            })

        out_df = pd.DataFrame(results)
        out_df.to_excel(EXCEL_OUTPUT, index=False)
        print(f"Processed rows {batch_start} to {batch_end-1}, partial results -> {EXCEL_OUTPUT}")

        batch_start = batch_end

    print("All done. Final file saved to:", EXCEL_OUTPUT)



In [ ]:
# JSON Prasing for Interventions

DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
INPUT_FILE = os.path.join(DATA_FOLDER, "ai_interventions.xlsx")
OUTPUT_FILE = os.path.join(DATA_FOLDER, "ai_interventions_parsed.xlsx")

def main():
    df = pd.read_excel(INPUT_FILE)

    drug_names_col = []
    dosages_col = []
    administration_route_col = []
    frequency_col = []
    therapy_class_col = []

    for _, row in df.iterrows():
        full_raw_response = row.get("full_raw_response", "")
        if not isinstance(full_raw_response, str) or not full_raw_response.strip():
            drug_names_col.append([])
            dosages_col.append([])
            administration_route_col.append("N/A")
            frequency_col.append("N/A")
            therapy_class_col.append("N/A")
            continue

        try:
            parsed_json = json.loads(full_raw_response)

            drug_names = parsed_json.get("drug_names", [])
            dosages = parsed_json.get("dosages", [])
            administration_route = parsed_json.get("administration_route", "N/A")
            frequency = parsed_json.get("frequency", "N/A")
            therapy_class = parsed_json.get("therapy_class", "N/A")

            drug_names_col.append(drug_names)
            dosages_col.append(dosages)
            administration_route_col.append(administration_route)
            frequency_col.append(frequency)
            therapy_class_col.append(therapy_class)

        except json.JSONDecodeError:
            drug_names_col.append([])
            dosages_col.append([])
            administration_route_col.append("N/A")
            frequency_col.append("N/A")
            therapy_class_col.append("N/A")

    df["drug_names_parsed"] = drug_names_col
    df["dosages_parsed"] = dosages_col
    df["administration_route_parsed"] = administration_route_col
    df["frequency_parsed"] = frequency_col
    df["therapy_class_parsed"] = therapy_class_col

    df.to_excel(OUTPUT_FILE, index=False)
    print(f"Parsing complete. Output saved to: {OUTPUT_FILE}")



In [ ]:
# JSON Prasing for Outcomes


DATA_FOLDER = "/Users/davidshevchenko/Documents/Github/Clinical-Trials-Dashboard/data"
INPUT_FILE = os.path.join(DATA_FOLDER, "ai_outcomes.xlsx")
OUTPUT_FILE = os.path.join(DATA_FOLDER, "ai_outcomes_parsed.xlsx")

def main():
    df = pd.read_excel(INPUT_FILE)

    key_variables_col = []
    time_frame_col = []
    outcome_category_col = []

    for _, row in df.iterrows():
        full_raw_response = row.get("full_raw_response", "")
        if not isinstance(full_raw_response, str) or not full_raw_response.strip():
            key_variables_col.append([])
            time_frame_col.append("N/A")
            outcome_category_col.append("N/A")
            continue

        try:
            parsed_json = json.loads(full_raw_response)

            key_variables = parsed_json.get("key_variables", [])
            time_frame = parsed_json.get("time_frame", "N/A")
            outcome_category = parsed_json.get("outcome_category", "N/A")

            key_variables_col.append(key_variables)
            time_frame_col.append(time_frame)
            outcome_category_col.append(outcome_category)

        except json.JSONDecodeError:
            key_variables_col.append([])
            time_frame_col.append("N/A")
            outcome_category_col.append("N/A")

    df["key_variables_parsed"] = key_variables_col
    df["time_frame_parsed"] = time_frame_col
    df["outcome_category_parsed"] = outcome_category_col

    df.to_excel(OUTPUT_FILE, index=False)
    print(f"Parsing complete. Output saved to: {OUTPUT_FILE}")

